In [ ]:
#This notebook detrends site-wise N₂O and O₃ time series, then computes and plots seasonal cycles (monthly means ± 95% CI) for each latitude zone and altitude.
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import calendar
from scipy import stats as ss

plt.rcParams['figure.dpi'] = 100
plt.rcParams['lines.markersize'] = 4
plt.rcParams['font.size'] = 14
plt.rcParams['axes.prop_cycle'] = plt.cycler('color', 'brgmyk')


In [ ]:
from pathlib import Path
# location of processed monthly means produced by the "monthly_averages_git" notebook
DATA_DIR = Path("./trends/monthly_averages")

In [ ]:
# read files with monthly average values
data_68 = pd.read_csv(DATA_DIR / 'monthly_averages_68_n2o.csv')
data_46 = pd.read_csv(DATA_DIR / 'monthly_averages_46_n2o.csv')
data_32 = pd.read_csv(DATA_DIR / 'monthly_averages_32_n2o.csv')
data_22 = pd.read_csv(DATA_DIR / 'monthly_averages_22_n2o.csv')

In [ ]:
o3_68 = pd.read_csv(DATA_DIR / 'monthly_averages_68_o3.csv')
o3_46 = pd.read_csv(DATA_DIR / 'monthly_averages_46_o3.csv')
o3_32 = pd.read_csv(DATA_DIR / 'monthly_averages_32_o3.csv')
o3_22 = pd.read_csv(DATA_DIR / 'monthly_averages_22_o3.csv')

In [ ]:
data_68 = data_68[data_68['count'] >= 5]
data_46 = data_46[data_46['count'] >= 5]
data_32 = data_32[data_32['count'] >= 5]
data_22 = data_22[data_22['count'] >= 5]

In [ ]:
o3_68 = o3_68[o3_68['count'] >= 5]
o3_46 = o3_46[o3_46['count'] >= 5]
o3_32 = o3_32[o3_32['count'] >= 5]
o3_22 = o3_22[o3_22['count'] >= 5]

In [ ]:
n2o_dfs = [data_22, data_32, data_46, data_68]
o3_dfs = [o3_22, o3_32, o3_46, o3_68]


In [ ]:
def add_detrended(df, date_column='date', group_col='location', 
                           value_col='mean_concentration', detrend_col='detrended_concentration'):
    """
    Adds a new column to the DataFrame that contains the values in value_col detrended
    by a linear fit performed separately for each group in group_col. The detrended values 
    are anchored at the center of the time series (so that the data remain on an absolute scale).
    
    NaN values in the value_col are ignored during the detrending calculation and remain NaN 
    in the resulting detrended column.
    
    Parameters:
      - df (DataFrame): Input DataFrame containing at least the date, group, and value columns.
      - date_column (str): Column name for the date. (Dates will be converted to datetime if needed.)
      - group_col (str): Column name on which to group the data (e.g. 'location').
      - value_col (str): Column name of the values to detrend (e.g. 'mean_concentration').
      - detrend_col (str): Name of the new column that will store the detrended values.
    
    Returns:
      - DataFrame: A copy of the original DataFrame with an additional column for detrended values.
    """
    # Create a copy of the dataframe to avoid modifying the original data.
    df = df.copy()
    
    # Ensure the date column is in datetime format.
    df[date_column] = pd.to_datetime(df[date_column])
    
    # Sort the DataFrame by group and date.
    df.sort_values([group_col, date_column], inplace=True)
    
    # Create the new column initialized with NaNs.
    df[detrend_col] = np.nan
    
    # Process each group separately.
    for loc, group in df.groupby(group_col):
        # Identify the valid (non-NaN) values.
        valid_mask = group[value_col].notna()
        if valid_mask.sum() < 2:
            # If there are fewer than 2 valid points, we can't fit a trend; skip this group.
            continue

        # Work only with the valid entries.
        sub = group.loc[valid_mask]
        # Convert dates to ordinal numbers for fitting.
        time_ord = sub[date_column].apply(lambda x: x.toordinal()).values
        values = sub[value_col].values
        
        # Fit a linear trend: values = a * time + b.
        coeffs = np.polyfit(time_ord, values, 1)
        
        # Define a baseline using the mean ordinal value so that the detrended data are 
        # adjusted relative to the center of the time period.
        t_center = time_ord.mean()
        baseline = np.polyval(coeffs, t_center)
        
        # Calculate the detrended values.
        # This subtracts the deviation of the trend from its baseline.
        detrended_values = values - (np.polyval(coeffs, time_ord) - baseline)
        
        # Assign the detrended values back to the original DataFrame.
        df.loc[sub.index, detrend_col] = detrended_values
    
    return df


In [ ]:
# Initialize a new list to hold the updated N2O DataFrames.
n2o_detrended = []

# Loop over each DataFrame, apply the detrending function, and append the result to the new list.
for df in n2o_dfs:
    new_df = add_detrended(df, 
                           date_column='date', 
                           group_col='location', 
                           value_col='mean_concentration', 
                           detrend_col='detrended_concentration')
    n2o_detrended.append(new_df)

In [ ]:
# Initialize a new list to hold the updated O3 DataFrames.
o3_detrended = []

# Loop over each DataFrame, apply the detrending function, and append the result to the new list.
for df in o3_dfs:
    new_df = add_detrended(df, 
                           date_column='date', 
                           group_col='location', 
                           value_col='mean_concentration', 
                           detrend_col='detrended_concentration')
    o3_detrended.append(new_df)

In [ ]:

def plot_monthly_means_by_latitude_zone_across_period(
    n2o_dfs: list,
    o3_dfs: list,
    zone_inds: list,
    display_names: dict,
    altitudes: list = ('22 hPa', '32 hPa', '46 hPa', '68 hPa'),
    #altitudes: list = ('22 hPa', '68 hPa'),
    value_col: str = 'detrended_concentration',
    date_col: str = 'date',
    zone_col: str = 'latitude_zone',
    confidence: float = 0.95,
    palette: list = None,
    save_path=None,
    save_dpi=300,
    zone_ci_alpha: dict = None,
    show_errorbar: bool = False
) -> None:
    """
    Plots mean detrended concentrations and 95% CI by month-of-year for chosen latitude zones
    across four altitudes (rows) and two species (cols: N2O left, O3 right).

    Y-limits are set per-subplot to a fixed window centered on that subplot's data:
      - Left column (N2O): 150 units
      - Right column (O3): 3 units
    If the data span exceeds the window, the axis expands to fit with a small pad.
    """
    default_palette = ['#377eb8', '#ff7f00', '#4daf4a', '#f781bf',
                       '#a65628', '#984ea3', '#999999', '#e41a1c', '#dede00']
    palette = palette or default_palette
    zone_ci_alpha = zone_ci_alpha or {}
    default_alpha = 0.3

    def prepare_monthly(df):
        df = df.copy()
        df[date_col] = pd.to_datetime(df[date_col])
        df['month'] = df[date_col].dt.month
        stats_df = (
            df.groupby([zone_col, 'month'])[value_col]
              .agg(['mean', 'std', 'count'])
              .reset_index()
        )
        alpha = 1 - confidence
        stats_df['ci_half'] = stats_df.apply(
            lambda r: ss.t.ppf(1 - alpha/2, df=r['count']-1) * r['std'] / (r['count']**0.5)
            if r['count'] > 1 else 0,
            axis=1
        )
        stats_df['ci_lower'] = stats_df['mean'] - stats_df['ci_half']
        stats_df['ci_upper'] = stats_df['mean'] + stats_df['ci_half']
        return stats_df

    stats_n2o = [prepare_monthly(df) for df in n2o_dfs]
    stats_o3  = [prepare_monthly(df) for df in o3_dfs]

    month_labels = list(calendar.month_abbr)[1:]
    fig, axes = plt.subplots(nrows=len(altitudes), ncols=2, figsize=(14, 4*len(altitudes)), sharex=True)

    for i, alt in enumerate(altitudes):
        for j, stats_list in enumerate((stats_n2o, stats_o3)):
            ax = axes[i, j]

            # plot all zones + collect per-subplot extents (use CI bounds so ribbons aren't clipped)
            ymins, ymaxs = [], []
            for idx, zone in enumerate(zone_inds):
                display = display_names.get(zone, str(zone))
                df_stats = stats_list[i]
                grp = df_stats[df_stats[zone_col] == zone].sort_values('month')
                if grp.empty:
                    continue

                color = palette[idx % len(palette)]
                a_ci = zone_ci_alpha.get(zone, default_alpha)

                ax.fill_between(grp['month'], grp['ci_lower'], grp['ci_upper'],
                                color=color, alpha=a_ci, zorder=1)
                if show_errorbar:
                    ax.errorbar(grp['month'], grp['mean'], yerr=grp['ci_half'],
                                fmt='none', ecolor=color, alpha=a_ci, capsize=3, zorder=2)
                ax.plot(grp['month'], grp['mean'], color=color, label=display, zorder=3)

                ymins.append(grp['ci_lower'].min())
                ymaxs.append(grp['ci_upper'].max())

            # title, grid
            ax.set_title(f"{alt}")
            ax.grid(True)

            # ---- centered, fixed-size windows per column ----
            # Change these two constants if you want different windows:
            window = 150 if j == 0 else 3    # N2O left / O3 right
            if ymins and ymaxs:
                data_min, data_max = min(ymins), max(ymaxs)
                spread = data_max - data_min
                center = 0.5 * (data_min + data_max)

                if spread >= window:
                    pad = 0.05 * spread
                    y0, y1 = data_min - pad, data_max + pad
                else:
                    half = window / 2.0
                    y0, y1 = center - half, center + half

                # keep O3 non-negative in constant-window case
                if j == 1 and spread < window and y0 < 0:
                    shift = -y0
                    y0, y1 = 0, y1 + shift

                ax.set_ylim(y0, y1)

            # y labels
            if j == 0:
                ax.set_ylabel("N₂O (ppbv)")
            else:
                ax.set_ylabel("O₃ (ppmv)")

            if i < len(altitudes)-1:
                ax.tick_params(labelbottom=False)

    # X-axis labels on bottom row
    for j in range(2):
        axes[-1, j].set_xticks(range(1, 13))
        axes[-1, j].set_xticklabels(month_labels, rotation=45)

    # Shared legend at bottom
    handles, labels = axes[0, 0].get_legend_handles_labels()
    fig.subplots_adjust(bottom=0.12, hspace=0.3)
    fig.legend(handles, labels, loc='lower center',
               ncol=len(zone_inds), fontsize=16, bbox_to_anchor=(0.5, 0.01))
    
    plt.tight_layout(rect=[0, 0.05, 1, 1])
    if save_path:
        plt.savefig(save_path, dpi=save_dpi)
    plt.show()


In [ ]:
plot_monthly_means_by_latitude_zone_across_period(
    n2o_dfs=n2o_detrended,
    o3_dfs=o3_detrended,
    zone_inds=[1,2,3],
    display_names={1:'Northern Hemisphere', 2:'Tropics', 3:'Southern Hemisphere'},
    show_errorbar=True,
    save_path = None
)